In [1]:
import tensorflow as tf
import utils
import numpy as np
import random as rn
import os

In [2]:
SEED = 123
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(SEED)
np.random.seed(SEED)
rn.seed(SEED)

data = np.load(os.path.join('.','data','royalty.npz'))

triples = data['triples']
traces = data['traces']
entities = data['entities'].tolist()
num_entities = len(entities)
relations = data['relations'].tolist()
num_relations = len(relations)
embedding_dim = 3
ent2idx = dict(zip(entities, range(num_entities)))
rel2idx = dict(zip(relations, range(num_relations)))

In [3]:
train2idx = utils.array2idx(triples, ent2idx,rel2idx)

In [4]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import RandomUniform
from tensorflow.python.ops import embedding_ops
import tensorflow.keras.backend as K

entity_embeddings = Embedding(
    input_dim=num_entities,
    output_dim=embedding_dim,
    name='entity_embeddings',
    embeddings_initializer=RandomUniform(
        minval=-1,
        maxval=1,
        seed=SEED
        )
    )

In [5]:
head_input = tf.keras.Input(shape=(), name='head_input',dtype=tf.int64)
rel_input = tf.keras.Input(shape=(), name='rel_input',dtype=tf.int64)
tail_input = tf.keras.Input(shape=(), name='tail_input',dtype=tf.int64)

In [6]:
head_e = entity_embeddings(head_input)
tail_e = entity_embeddings(tail_input)

In [7]:
class RGCN_Layer(tf.keras.layers.Layer):
    def __init__(self,num_relations,output_dim,**kwargs):
        super(RGCN_Layer,self).__init__(**kwargs)
        self.num_relations = num_relations
        self.output_dim = output_dim
    
    def build(self,input_shape):

        in_shape = input_shape[-1][-1]

        self.W_r = self.add_weight(
            shape=(self.num_relations,self.output_dim,in_shape),
            trainable=True,
            initializer="random_normal",
            name='W_r'
        )
        
        self.W0 = self.add_weight(
            shape=(self.output_dim,in_shape),
            trainable=True,
            initializer='random_normal',
            name='W0'
        )
        
    def call(self,inputs):
        
        head_input,tail_input = inputs
        
        tail_update = tf.matmul(self.W_r,tail_input,transpose_b=True)
        
        head_update = tf.matmul(self.W0,head_input,transpose_b=True)
        
        update = tf.reduce_sum(tail_update + head_update, axis=0)

        return tf.transpose(update)

In [8]:
class DistMult(tf.keras.layers.Layer):
    def __init__(self, num_relations,**kwargs):
        super(DistMult,self).__init__(**kwargs)
        self.num_relations = num_relations
        
    def build(self,input_shape):
        
        embedding_dim = input_shape[0][-1]
        
        self.kernel = self.add_weight(
            shape=(self.num_relations,embedding_dim),
            trainable=True,
            initializer='random_normal',
            name='rel_embedding'
        )
        
    def call(self,inputs):
        
        head_e,rel_idx,tail_e = inputs
        
        rel_e = embedding_ops.embedding_lookup_v2(self.kernel,rel_idx)
        
        return tf.reduce_sum(head_e*rel_e*tail_e, axis=-1)

In [9]:
new_head = RGCN_Layer(num_relations=num_relations,output_dim=5)([head_e,tail_e])
new_head = tf.keras.layers.Activation('sigmoid')(new_head)

new_tail = RGCN_Layer(num_relations=num_relations,output_dim=5)([tail_e,head_e])
new_tail = tf.keras.layers.Activation('sigmoid')(new_tail)

output = DistMult(num_relations=num_relations)([new_head,rel_input,new_tail])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

In [10]:
#model = tf.keras.Model([head_input,rel_input,tail_input],[output])

In [11]:
#model.compile(loss='binary_crossentropy', optimizer='sgd')

In [12]:
#model.fit(x=[train2idx[:,0],train2idx[:,1],train2idx[:,2]],y=np.ones(train2idx.shape[0]),epochs=1)

In [13]:
class RGCN_Model(tf.keras.Model):
    def __init__(self,num_entities,*args,**kwargs):
        super(RGCN_Model,self).__init__(*args, **kwargs)
        self.num_entities = num_entities
        
    def train_step(self,data):

        pos_head,rel,pos_tail = data[0]
        y = data[1]
        
        neg_head, neg_tail = utils.get_negative_triples(
            head=pos_head, 
            rel=rel, 
            tail=pos_tail,
            num_entities=self.num_entities
            )
        
        head = tf.concat([pos_head,neg_head],axis=0)
        rel = tf.concat([rel,rel],axis=0)
        tail = tf.concat([pos_tail,neg_tail],axis=0)
        
        y_neg = tf.zeros_like(y)
        
        y = tf.concat([y,y_neg],axis=0)
        
        with tf.GradientTape() as tape:
            
            y_pred = self([head,rel,tail],training=True)
            
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)
            
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        self.compiled_metrics.update_state(y, y_pred)

        return {m.name: m.result() for m in self.metrics}

In [14]:
model = RGCN_Model(inputs=[head_input,rel_input,tail_input],outputs=[output],num_entities=num_entities)

In [15]:
model.compile(loss='binary_crossentropy', optimizer='sgd')

In [16]:
model.fit(x=[train2idx[:,0],train2idx[:,1],train2idx[:,2]],y=np.ones(train2idx.shape[0]),epochs=2000,verbose=0)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [17]:
# import dgl
# import torch as th
# from dgl.nn import RelGraphConv
# g = dgl.graph(([0,1,2,3,2,5,0], [1,2,3,4,0,3,1]))
# conv = RelGraphConv(10, 5, 3, regularizer='basis')
# feat = th.ones(6, 10)
# etype = th.tensor(np.array([0,1,2,0,1,2,1]).astype(np.int64))
# conv(g, feat, etype)

In [24]:
(model.predict([train2idx[:,0],train2idx[:,1],train2idx[:,2]]) > .5).sum()/train2idx.shape[0]

0.734378401915959

In [20]:
train2idx

array([ 583, 5732,  781, ..., 4703, 2114, 1917])

In [ ]:
#for triple i:
    #get k hop subgraph?
    #compute pred 
    #define masks
    #optimize loss, return masks

In [21]:
def get_adjlist(train2idx,entities):

    adj_list = [[] for _ in entities]

    for i,triplet in enumerate(train2idx):
        adj_list[triplet[0]].append([i, triplet[2]])
        adj_list[triplet[2]].append([i, triplet[0]])

    degrees = np.array([len(a) for a in adj_list])
    adj_list = [np.array(a) for a in adj_list]
    
    return adj_list,degrees

In [23]:
#get_adjlist(train2idx,entities)